In [1]:
import libsql_client
from app.config.settings import api_settings
import requests
import time

In [2]:
cursor = libsql_client.create_client_sync(
    url=api_settings.TURSO_API_URL,
    auth_token=api_settings.TURSO_API_TOKEN
)

In [3]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS anime (
            id INTEGER PRIMARY KEY,
            title TEXT,
            url TEXT
               )
""")

In [4]:
requests_per_minute = 60
requests_per_second = 3

In [5]:
base_url = "https://api.jikan.moe/v4/anime"

In [6]:
def fetch_and_store_data(page):
    url = f"{base_url}?page={page}"
    response = requests.get(url)
    data = response.json()
    animes = []
    for item in data["data"]:
        obj= {
            "id": item["mal_id"],
            "title": item["title"],
            "url": item["url"]
        }
        anime_id = item["mal_id"]
        title = item["title"]
        url = item["url"]
        animes.append(obj)

        # # Verifica si el anime ya existe en la base de datos
        rs = cursor.execute("SELECT id FROM anime WHERE id = ?", [anime_id])
        # existing_row = cursor.fetchone()
        existing_row = len(rs.rows) > 0

        if not existing_row:
            # Inserta el anime en la base de datos si no existe
            cursor.execute("INSERT INTO anime (id, title,url) VALUES (:id, :title, :url)", 
                           {
                                 "id": anime_id,
                                 "title": title,
                                 "url": url
                           })
    return animes


In [7]:
total_pages = 25481 // 25  # Calcula el número total de páginas
for page in range(1, total_pages + 1):
    data = fetch_and_store_data(page)
    
    # Calcula el tiempo de espera entre las solicitudes para respetar el límite por segundo
    if page % requests_per_second == 0:
        time.sleep(1)

    # Calcula el tiempo de espera para respetar el límite por minuto
    if page % (requests_per_minute // requests_per_second) == 0:
        time.sleep(60)

# Cierra la conexión a la base de datos
cursor.close()